In [20]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn import metrics
from sklearn.metrics import classification_report, f1_score
from catboost import CatBoostClassifier

from load_dataset import load_data

In [21]:
dict_accuracy = {}
dict_f1 = {}

# Instancier jeux de données

Depuis la classe utilitaire load_dataset, instancier différentes versions du jeux de données. Les modèles seront testés sur la base des 4 jeux de données, ce qui permettra d'avoir une meilleure visibilité sur la pertinence des modifications de chaque versions en rapport à leurs résultats respectifs.

Cela facilitera aussi la sélection des jeux de données à pousser plus loin en analyse des features.

La variables cat_features_catb est surtout utile pour l'entraînement du CatBoost qui ne peux seulement consommer que la version Simplified du dataset.

In [22]:
X, y, cat_features = load_data().get_data_X_y(data='simplify', OneHot=True)
X_train_simplified_oneHot, X_test_simplified_oneHot, y_train_simplified_oneHot, y_test_simplified_oneHot = train_test_split(X, y, test_size=0.3, random_state=42)

In [23]:
X, y, cat_features = load_data().get_data_X_y(data='original', OneHot=True)
X_train_original_oneHot, X_test_original_oneHot, y_train_original_oneHot, y_test_original_oneHot = train_test_split(X, y, test_size=0.3, random_state=42)

In [24]:
X, y, cat_features_catb = load_data().get_data_X_y(data='simplify')
X_train_simplified, X_test_simplified, y_train_simplified, y_test_simplified = train_test_split(X, y, test_size=0.3, random_state=42)

In [25]:
X, y, cat_features = load_data().get_data_X_y(data='original')
X_train_original, X_test_original, y_train_original, y_test_original = train_test_split(X, y, test_size=0.3, random_state=42)

# XBboost (Avec One-Hot seulement car SHAP ne supporte pas encore XGBoost gérant les données catégoriques)

## Recherche en grille

Le booster est, ici, nécessairement Gblinear. Les deux autre choix, en l'occurence Gbtree et Dart, prenaient un temps immense à être entrainés dans la recherche en grille. De plus, des tests hors recherche en grille ont prouvés à plusieurs reprises que Gblinear offre des meilleurs résultats d'accuracy et de score f1 sur nos données.

Par mesure de simplicité et d'efficacité, le paramètre booster est donc défini par défaut sur Gblinear et la recherche en grille s'effectue sur la base des paramètres relatifs à un booster de type Gblinear.

In [26]:
# init xgb classifier
xgb_clf = xgb.XGBClassifier(verbosity=3, random_state=42)

param_grid = {
    'objective':["binary:logistic", "binary:hinge"],
    'booster':["gblinear"],
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.05, 0.1],
}


# init grid search
grid_search = GridSearchCV(estimator=xgb_clf, param_grid=param_grid, cv=3, scoring='f1')

grid_search.fit(X_train_simplified_oneHot, y_train_simplified_oneHot)

best_parameters = grid_search.best_params_
best_score = grid_search.best_score_

print("Meilleurs hyperparamèetres:")
best_parameters_df = {key: [value] for key, value in best_parameters.items()}
df = pd.DataFrame(best_parameters_df)
print(df)
print()
print("Meilleur score:", best_score)

[10:00:14] ======== Monitor (0): Learner ========
[10:00:14] Configure: 0.000117s, 1 calls @ 117us

[10:00:14] EvalOneIter: 0.000208s, 50 calls @ 208us

[10:00:14] GetGradient: 0.001593s, 50 calls @ 1593us

[10:00:14] PredictRaw: 0.00688s, 50 calls @ 6880us

[10:00:14] UpdateOneIter: 0.038591s, 50 calls @ 38591us

[10:00:14] ======== Monitor (0): GBLinear ========
[10:00:14] DoBoost: 0.029913s, 50 calls @ 29913us

[10:00:14] PredictBatch: 0.006856s, 50 calls @ 6856us

[10:00:14] PredictBatchInternal: 0.006839s, 50 calls @ 6839us

[10:00:14] ======== Monitor (0): Learner ========
[10:00:14] Configure: 0.000124s, 1 calls @ 124us

[10:00:14] EvalOneIter: 0.000247s, 50 calls @ 247us

[10:00:14] GetGradient: 0.001881s, 50 calls @ 1881us

[10:00:14] PredictRaw: 0.007441s, 50 calls @ 7441us

[10:00:14] UpdateOneIter: 0.044546s, 50 calls @ 44546us

[10:00:14] ======== Monitor (0): GBLinear ========
[10:00:14] DoBoost: 0.034981s, 50 calls @ 34981us

[10:00:14] PredictBatch: 0.007415s, 50 calls 

[10:00:14] ======== Monitor (0): Learner ========
[10:00:14] Configure: 0.00021s, 1 calls @ 210us

[10:00:14] EvalOneIter: 0.000249s, 50 calls @ 249us

[10:00:14] GetGradient: 0.002225s, 50 calls @ 2225us

[10:00:14] PredictRaw: 0.007161s, 50 calls @ 7161us

[10:00:14] UpdateOneIter: 0.051499s, 50 calls @ 51499us

[10:00:14] ======== Monitor (0): GBLinear ========
[10:00:14] DoBoost: 0.041774s, 50 calls @ 41774us

[10:00:14] PredictBatch: 0.007126s, 50 calls @ 7126us

[10:00:14] PredictBatchInternal: 0.007101s, 50 calls @ 7101us

[10:00:14] ======== Monitor (0): Learner ========
[10:00:14] Configure: 0.000192s, 1 calls @ 192us

[10:00:14] EvalOneIter: 0.000325s, 50 calls @ 325us

[10:00:14] GetGradient: 0.003203s, 50 calls @ 3203us

[10:00:14] PredictRaw: 0.04314s, 50 calls @ 43140us

[10:00:14] UpdateOneIter: 0.092693s, 50 calls @ 92693us

[10:00:14] ======== Monitor (0): GBLinear ========
[10:00:14] DoBoost: 0.046016s, 50 calls @ 46016us

[10:00:14] PredictBatch: 0.043101s, 50 calls 

In [27]:
xgb_acc = {}
xgb_f1 = {}

Meilleur modèle XGBoost:

In [28]:
xgbClassifier_optimized = xgb.XGBClassifier(**best_parameters, random_state=42)

## Simplified + One hot

In [29]:
xgbClassifier_optimized.fit(X_train_simplified_oneHot, y_train_simplified_oneHot)
xgbClassifier_optimized_acc = xgbClassifier_optimized.score(X_test_simplified_oneHot, y_test_simplified_oneHot)
xgbClassifier_optimized_predz = xgbClassifier_optimized.predict(X_test_simplified_oneHot)
xgbClassifier_optimized_f1 = f1_score(y_test_simplified_oneHot, xgbClassifier_optimized_predz, average='binary')

xgb_acc["Simplified + OneHot"] = xgbClassifier_optimized_acc
xgb_f1["Simplified + OneHot"] = xgbClassifier_optimized_f1

## Original + One hot

In [30]:
xgbClassifier_optimized.fit(X_train_original_oneHot, y_train_original_oneHot)
xgbClassifier_optimized_acc = xgbClassifier_optimized.score(X_test_original_oneHot, y_test_original_oneHot)
xgbClassifier_optimized_predz = xgbClassifier_optimized.predict(X_test_original_oneHot)
xgbClassifier_optimized_f1 = f1_score(y_test_original_oneHot, xgbClassifier_optimized_predz, average='binary')

xgb_acc["Original + OneHot"] = xgbClassifier_optimized_acc
xgb_f1["Original + OneHot"] = xgbClassifier_optimized_f1

## Simplified

In [31]:

xgb_acc["Simplified"] = None
xgb_f1["Simplified"] = None

## Original

In [32]:
xgb_acc["Original"] = None
xgb_f1["Original"] = None

## Aggrégation

In [33]:
dict_accuracy["XGboost"] = xgb_acc
dict_f1["XGboost"] = xgb_f1

# SVC

## Recherche en grille

Pour le SVC, le choix du kernel doit obligatoirement être linéaire, puisque cela empêche au modèle d'augmenter la dimensionnalité des données, ce qui nous empêcherait plus tard de faire une analyse claire des features les plus prenants pour la classification.

L'utilisation d'un noyeau linéaire permet au SVM de rester dans l'espace dimensionnel de base et ne projète pas les features dans un espace à plus haute dimensionnalité, comme le ferait un noyeau Gaussien ou RBF, par exemple. C'est pourquoi nous pourrons directement récupérer l'importance des features plus tard dans l'analyse.

In [34]:
# by default SVC
svcClassifier = SVC(kernel="rbf", C=1.0, degree=3, gamma='scale', random_state=42)

param_grid = {
    'kernel': ['linear'],
    'C': [0.1, 1, 10, 50],
    'gamma': ['scale', 0.001, 0.1, 1],
    'class_weight': [None, 'balanced']
}

# init grid search
grid_search = GridSearchCV(estimator=svcClassifier, param_grid=param_grid, cv=3, scoring='f1', verbose=3)

grid_search.fit(X_train_simplified_oneHot, y_train_simplified_oneHot)

best_parameters = grid_search.best_params_
best_score = grid_search.best_score_

print("Meilleurs hyperparamèetres:")
best_parameters_df = {key: [value] for key, value in best_parameters.items()}
df = pd.DataFrame(best_parameters_df)
print(df)
print()
print("Meilleur score:", best_score)

Fitting 3 folds for each of 32 candidates, totalling 96 fits
[CV 1/3] END C=0.1, class_weight=None, gamma=scale, kernel=linear;, score=0.929 total time=   0.2s
[CV 2/3] END C=0.1, class_weight=None, gamma=scale, kernel=linear;, score=0.932 total time=   0.2s
[CV 3/3] END C=0.1, class_weight=None, gamma=scale, kernel=linear;, score=0.923 total time=   0.1s
[CV 1/3] END C=0.1, class_weight=None, gamma=0.001, kernel=linear;, score=0.929 total time=   0.1s
[CV 2/3] END C=0.1, class_weight=None, gamma=0.001, kernel=linear;, score=0.932 total time=   0.1s
[CV 3/3] END C=0.1, class_weight=None, gamma=0.001, kernel=linear;, score=0.923 total time=   0.1s
[CV 1/3] END C=0.1, class_weight=None, gamma=0.1, kernel=linear;, score=0.929 total time=   0.1s
[CV 2/3] END C=0.1, class_weight=None, gamma=0.1, kernel=linear;, score=0.932 total time=   0.1s
[CV 3/3] END C=0.1, class_weight=None, gamma=0.1, kernel=linear;, score=0.923 total time=   0.1s
[CV 1/3] END C=0.1, class_weight=None, gamma=1, kernel

In [35]:
svc_acc = {}
svc_f1 = {}

Meilleur modèle SVC:

In [36]:
svcClassifier_optimized = SVC(**best_parameters, random_state=42)

## Simplified + One hot

In [37]:
svcClassifier_optimized.fit(X_train_simplified_oneHot, y_train_simplified_oneHot)
svcClassifier_optimized_acc = svcClassifier_optimized.score(X_test_simplified_oneHot, y_test_simplified_oneHot)
svcClassifier_optimized_predz = svcClassifier_optimized.predict(X_test_simplified_oneHot)
svcClassifier_optimized_f1 = f1_score(y_test_simplified_oneHot, svcClassifier_optimized_predz, average='binary')

svc_acc["Simplified + OneHot"] = svcClassifier_optimized_acc
svc_f1["Simplified + OneHot"] = svcClassifier_optimized_f1

## Original + One hot

In [38]:
svcClassifier_optimized.fit(X_train_original_oneHot, y_train_original_oneHot)
svcClassifier_optimized_acc = svcClassifier_optimized.score(X_test_original_oneHot, y_test_original_oneHot)
svcClassifier_optimized_predz = svcClassifier_optimized.predict(X_test_original_oneHot)
svcClassifier_optimized_f1 = f1_score(y_test_original_oneHot, svcClassifier_optimized_predz, average='binary')

svc_acc["Original + OneHot"] = svcClassifier_optimized_acc
svc_f1["Original + OneHot"] = svcClassifier_optimized_f1

## Simplified

In [39]:
svcClassifier_optimized.fit(X_train_simplified, y_train_simplified)
svcClassifier_optimized_acc = svcClassifier_optimized.score(X_test_simplified, y_test_simplified)
svcClassifier_optimized_predz = svcClassifier_optimized.predict(X_test_simplified)
svcClassifier_optimized_f1 = f1_score(y_test_simplified, svcClassifier_optimized_predz, average='binary')

svc_acc["Simplified"] = svcClassifier_optimized_acc
svc_f1["Simplified"] = svcClassifier_optimized_f1

## Original

In [40]:
svcClassifier_optimized.fit(X_train_original, y_train_original)
svcClassifier_optimized_acc = svcClassifier_optimized.score(X_test_original, y_test_original)
svcClassifier_optimized_predz = svcClassifier_optimized.predict(X_test_original)
svcClassifier_optimized_f1 = f1_score(y_test_original, svcClassifier_optimized_predz, average='binary')

svc_acc["Original"] = svcClassifier_optimized_acc
svc_f1["Original"] = svcClassifier_optimized_f1

## Aggrégation

In [41]:
dict_accuracy["SVC"] = svc_acc
dict_f1["SVC"] = svc_f1

# KNN

## Recherche en grille

Le modèle par KNN n'accèpte pas d'argument random_state pour gérer la consistense du modèle.

In [42]:
n_range = 30
mean_acc = np.zeros(n_range)
for i in range(1,n_range + 1):
    #Train Model and Predict
    knn = KNeighborsClassifier(n_neighbors = i).fit(X_train_simplified_oneHot, y_train_simplified_oneHot)
    yhat= knn.predict(X_test_simplified_oneHot)
    mean_acc[i-1] = metrics.accuracy_score(y_test_simplified_oneHot, yhat)
k_range_cnt = 10
k_range = np.argpartition(mean_acc, -k_range_cnt)[-k_range_cnt:]

# by default knn
knnClassifier = KNeighborsClassifier()

param_grid = {
               'n_neighbors' : k_range,
               'weights' : ['uniform','distance'],
               'metric' : ['minkowski','euclidean','manhattan'],
               'algorithm' : ['ball_tree', 'kd_tree', 'brute'],
               'p' : [1,2],
}

# init grid search
grid_search = GridSearchCV(estimator=knnClassifier, param_grid=param_grid, cv=3, scoring='f1', verbose=3)

grid_search.fit(X_train_simplified_oneHot, y_train_simplified_oneHot)

best_parameters = grid_search.best_params_
best_score = grid_search.best_score_

print("Meilleurs hyperparamèetres:")
best_parameters_df = {key: [value] for key, value in best_parameters.items()}
df = pd.DataFrame(best_parameters_df)
print(df)
print()
print("Meilleur score:", best_score)

Fitting 3 folds for each of 360 candidates, totalling 1080 fits
[CV 1/3] END algorithm=ball_tree, metric=minkowski, n_neighbors=18, p=1, weights=uniform;, score=0.893 total time=   0.3s
[CV 2/3] END algorithm=ball_tree, metric=minkowski, n_neighbors=18, p=1, weights=uniform;, score=0.905 total time=   0.2s
[CV 3/3] END algorithm=ball_tree, metric=minkowski, n_neighbors=18, p=1, weights=uniform;, score=0.893 total time=   0.2s
[CV 1/3] END algorithm=ball_tree, metric=minkowski, n_neighbors=18, p=1, weights=distance;, score=0.894 total time=   0.2s
[CV 2/3] END algorithm=ball_tree, metric=minkowski, n_neighbors=18, p=1, weights=distance;, score=0.907 total time=   0.2s
[CV 3/3] END algorithm=ball_tree, metric=minkowski, n_neighbors=18, p=1, weights=distance;, score=0.896 total time=   0.2s
[CV 1/3] END algorithm=ball_tree, metric=minkowski, n_neighbors=18, p=2, weights=uniform;, score=0.896 total time=   0.2s
[CV 2/3] END algorithm=ball_tree, metric=minkowski, n_neighbors=18, p=2, weight

In [43]:
knn_acc = {}
knn_f1 = {}

Meilleur modèle KNN:

In [44]:
knnClassifier_optimized = KNeighborsClassifier(**best_parameters)

## Simplified + One hot

In [45]:
knnClassifier_optimized.fit(X_train_simplified_oneHot, y_train_simplified_oneHot)
knnClassifier_optimized_acc = knnClassifier_optimized.score(X_test_simplified_oneHot, y_test_simplified_oneHot)
knnClassifier_optimized_predz = knnClassifier_optimized.predict(X_test_simplified_oneHot)
knnClassifier_optimized_f1 = f1_score(y_test_simplified_oneHot, knnClassifier_optimized_predz, average='binary')

knn_acc["Simplified + OneHot"] = knnClassifier_optimized_acc
knn_f1["Simplified + OneHot"] = knnClassifier_optimized_f1

## Original + One hot

In [46]:
knnClassifier_optimized.fit(X_train_original_oneHot, y_train_original_oneHot)
knnClassifier_optimized_acc = knnClassifier_optimized.score(X_test_original_oneHot, y_test_original_oneHot)
knnClassifier_optimized_predz = knnClassifier_optimized.predict(X_test_original_oneHot)
knnClassifier_optimized_f1 = f1_score(y_test_original_oneHot, knnClassifier_optimized_predz, average='binary')

knn_acc["Original + OneHot"] = knnClassifier_optimized_acc
knn_f1["Original + OneHot"] = knnClassifier_optimized_f1

## Simplified

In [47]:
knnClassifier_optimized.fit(X_train_simplified, y_train_simplified)
knnClassifier_optimized_acc = knnClassifier_optimized.score(X_test_simplified, y_test_simplified)
knnClassifier_optimized_predz = knnClassifier_optimized.predict(X_test_simplified)
knnClassifier_optimized_f1 = f1_score(y_test_simplified, knnClassifier_optimized_predz, average='binary')

knn_acc["Simplified"] = knnClassifier_optimized_acc
knn_f1["Simplified"] = knnClassifier_optimized_f1

## Original

In [48]:
knnClassifier_optimized.fit(X_train_original, y_train_original)
knnClassifier_optimized_acc = knnClassifier_optimized.score(X_test_original, y_test_original)
knnClassifier_optimized_predz = knnClassifier_optimized.predict(X_test_original)
knnClassifier_optimized_f1 = f1_score(y_test_original, knnClassifier_optimized_predz, average='binary')

knn_acc["Original"] = knnClassifier_optimized_acc
knn_f1["Original"] = knnClassifier_optimized_f1

## Aggrégation

In [49]:
dict_accuracy["KNN"] = knn_acc
dict_f1["KNN"] = knn_f1

# CatBoost

## Recherche en grille

In [50]:
# Create a CatBoostClassifier
catboost_model = CatBoostClassifier(cat_features=cat_features_catb, verbose=0, random_state=42)

# Define the parameter grid to search
''' param_grid = {
    'iterations': [None, 100, 200, 300],
    'learning_rate': [None, 0.01, 0.05, 0.1],
    'depth': [None, 4, 6, 8],
    'l2_leaf_reg': [None, 1, 3, 5],
} '''
param_grid = {
    'iterations': [1000],
    'learning_rate': [0.01, 0.05, 0.1],
    'depth': [4, 6, 9],
    'l2_leaf_reg': [1, 3, 5],
    'one_hot_max_size': [2, 150]
}

# init grid search
grid_search = GridSearchCV(estimator=catboost_model, param_grid=param_grid, scoring='f1', cv=3, n_jobs=1, verbose=3)

grid_search.fit(X_train_simplified, y_train_simplified)

best_parameters = grid_search.best_params_
best_score = grid_search.best_score_

print("Meilleurs hyperparamèetres:")
best_parameters_df = {key: [value] for key, value in best_parameters.items()}
df = pd.DataFrame(best_parameters_df)
print(df)
print()
print("Meilleur score:", best_score)


Fitting 3 folds for each of 54 candidates, totalling 162 fits
[CV 1/3] END depth=4, iterations=1000, l2_leaf_reg=1, learning_rate=0.01, one_hot_max_size=2;, score=0.926 total time=   2.3s
[CV 2/3] END depth=4, iterations=1000, l2_leaf_reg=1, learning_rate=0.01, one_hot_max_size=2;, score=0.935 total time=   2.4s
[CV 3/3] END depth=4, iterations=1000, l2_leaf_reg=1, learning_rate=0.01, one_hot_max_size=2;, score=0.918 total time=   2.6s
[CV 1/3] END depth=4, iterations=1000, l2_leaf_reg=1, learning_rate=0.01, one_hot_max_size=150;, score=0.923 total time=   0.9s
[CV 2/3] END depth=4, iterations=1000, l2_leaf_reg=1, learning_rate=0.01, one_hot_max_size=150;, score=0.933 total time=   0.9s
[CV 3/3] END depth=4, iterations=1000, l2_leaf_reg=1, learning_rate=0.01, one_hot_max_size=150;, score=0.919 total time=   0.9s
[CV 1/3] END depth=4, iterations=1000, l2_leaf_reg=1, learning_rate=0.05, one_hot_max_size=2;, score=0.922 total time=   2.5s
[CV 2/3] END depth=4, iterations=1000, l2_leaf_reg

In [51]:
catb_acc = {}
catb_f1 = {}

Meilleur modèle CatB:

In [52]:
catbClassifier_optimized = grid_search.best_estimator_

## Simplified + One hot

In [53]:
catb_acc["Simplified + OneHot"] =  None
catb_f1["Simplified + OneHot"] = None

## Original + One hot

In [54]:
catb_acc["Original + OneHot"] = None
catb_f1["Original + OneHot"] = None

## Simplified

In [55]:
catbClassifier_optimized.fit(X_train_simplified, y_train_simplified)
catbClassifier_optimized_acc = catbClassifier_optimized.score(X_test_simplified, y_test_simplified)
catbClassifier_optimized_predz = catbClassifier_optimized.predict(X_test_simplified)
catbClassifier_optimized_f1 = f1_score(y_test_simplified, catbClassifier_optimized_predz, average='binary')

catb_acc["Simplified"] = catbClassifier_optimized_acc
catb_f1["Simplified"] = catbClassifier_optimized_f1

## Original

In [57]:
catb_acc["Original"] = None
catb_f1["Original"] = None

## Aggrégation

In [58]:
dict_accuracy["CatBoost"] = catb_acc
dict_f1["CatBoost"] = catb_f1

# Résultats

In [59]:
df_acc = pd.DataFrame(dict_accuracy)
df_acc

,XGboost,SVC,KNN,CatBoost
Simplified + OneHot,0.90450,0.916437,0.878788,NaN
Original + OneHot,0.89899,0.912764,0.876951,NaN
Simplified,NaN,0.915519,0.864096,0.913682
Original,NaN,0.918274,0.845730,NaN


In [60]:
df_f1 = pd.DataFrame(dict_f1)
df_f1

,XGboost,SVC,KNN,CatBoost
Simplified + OneHot,0.925287,0.934954,0.908966,NaN
Original + OneHot,0.921875,0.932288,0.907840,NaN
Simplified,NaN,0.934566,0.899183,0.932953
Original,NaN,0.936745,0.886792,NaN
